In [1]:
import sys
sys.path.append('C:/Users/manya/Documents/Ironhack/Course/Final-project/src')
from geo_json import get_hsz, geocode
import sql
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image 
import seaborn as sns 
import keras.models
from keras.preprocessing import image
from CNN_model import Image_Data_Generator_train
import numpy as np

## OUTPUT MODELO

In [2]:
model = keras.models.load_model('../models/model_def.keras')
train_generator = Image_Data_Generator_train('../images_15/train',(224,224), 32)
def predict_label (test_image_path):
     # 1. Load and preprocess the image
    test_img = image.load_img(test_image_path, target_size=(224, 224))
    test_img_arr = image.img_to_array(test_img)/255.0
    test_img_input = test_img_arr.reshape((1, test_img_arr.shape[0], test_img_arr.shape[1], test_img_arr.shape[2]))

    # 2. Make Predictions
    class_map = dict([(v, k) for k, v in train_generator.class_indices.items()])
    predicted_label = np.argmax(model.predict(test_img_input))
    predicted_vegetable = class_map[predicted_label]

    return predicted_vegetable

Found 15000 images belonging to 15 classes.


In [17]:
vegetable = predict_label("C:/Users/manya/OneDrive/Imágenes/Álbum de cámara/WIN_20230716_20_11_57_Pro.jpg")

1/1 [==============================] - 0s 61ms/step


In [18]:
print(vegetable)

Pumpkin


### OUTPUT LOCATION

In [5]:
coordenadas = geocode('calle pamplona ,21, Barcelona')
hsz_ = list(get_hsz(coordenadas[0],coordenadas[1]))[0]
print(hsz_)

10a


### Connection Database 

In [6]:
characteristics = sql.get_charactersitics(vegetable)
seasonality = sql.get_seasonality(vegetable,hsz_)

In [7]:
characteristics

,name,sowing,spacing,harvest,compatibility,avoid,image_link,temperature
0,Tomato,"Grow in seed trays, and plant out in 4-6 week...",40 - 60 cm apart,Harvest in 8-17 weeks.,"Asparagus, Chervil,Carrot, Celery, Chives, Pa...","Rosemary, Potatoes, Fennel, Cucumber",www.gardenate.com/image/26,16°C and 35°C


In [8]:
seasonality.isna().sum()

Jan    2
Feb    2
Mar    1
Apr    1
May    3
Jun    3
Jul    3
Aug    3
Sep    2
Oct    3
Nov    3
Dec    3
dtype: int64

In [12]:
def color_cells(val):
    if val == 'S':
        color = '#A68464'  # Color para 'S'
    elif val == 'T':
         color = '#A4AC86'  # Color para 'T'
    elif val == 'P':
        color = '#B6AD90'  # Color para 'P'
    else:
        color = 'white'  # Color para celdas vacías
    return f'background-color: {color}'
    
styled_cal = seasonality.style.applymap(color_cells)

In [13]:
styled_cal

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,S,S,None,None,None,None,None,None,None,None,None,None
1,None,None,T,T,None,None,None,None,None,None,None,None
2,None,None,P,P,None,None,None,None,P,None,None,None


In [14]:
def obtain_values (df):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    values = ['S', 'T', 'P']
    values_description = {'S': 'Plant undercover seed trays',
                           'T': 'Transplant seedlings',
                           'P': 'Plant seeds'}
    dict_ = {}

    for value in values:
        months_value = [month for month in months if df[month].str.contains(value).any()]
        if months_value:
            dict_[values_description[value]] = months_value

    return dict_

In [15]:
obtain_values(seasonality)

{'Plant undercover seed trays': ['Jan', 'Feb'],
 'Transplant seedlings': ['Mar', 'Apr'],
 'Plant seeds': ['Mar', 'Apr', 'Sep']}